<a href="https://colab.research.google.com/github/mvkhachaturyan/First/blob/main/FirstNote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

import pandas as pd
import numpy as np

# Загрузка ваших данных

my_data = pd.read_csv('/content/data.csv', header=None)

# Переопредели формат даты и времени
my_data[1] = pd.to_datetime(my_data[1], format='%d/%m/%y %H:%M:%S')

# Преобразование даты и времени в числовой формат (временные метки)
my_data[1] = my_data[1].apply(lambda x: x.timestamp())

# Проверь результат
print(my_data.head())

# Преобразование данных в numpy array и удостоверься, что все данные целочисленные
my_data = my_data.values

my_labels = pd.read_csv('/content/labels.csv', header=None).values

# Подготовка данных для обучения
X = my_data  # теперь включает порядковый номер и дату события в числовом формате
y1 = my_labels[:, :4]  # первые 4 колонки как первый набор меток
y2 = my_labels[:, 4:]  # последние 4 колонки как второй набор меток

# Проверьте размерности y1 и y2 здесь, чтобы удостовериться, что они правильные
print(y1.shape)
print(y2.shape)

# Определение модели
inputs = layers.Input(shape=(X.shape[1],))  # Обновление формы входа, чтобы соответствовать новому размеру X

# Слой Embedding для трансформера
max_value = np.max(my_data[:, 1])
embedding_layer = layers.Embedding(input_dim=int(max_value) + 1, output_dim=16)
transformer_input = embedding_layer(inputs)
transformer_input = layers.Dropout(0.2)(transformer_input)

# Слой трансформера
query_value_attention_seq = layers.MultiHeadAttention(num_heads=2, key_dim=2)(transformer_input, transformer_input)
transformer_output = layers.GlobalAveragePooling1D()(query_value_attention_seq)

# Плотные слои для финального предсказания
dense_output = layers.Dense(32, activation='relu')(transformer_output)
output1 = layers.Dense(4, name='output1')(dense_output)  # Первый выходной слой с 4 нейронами
output2 = layers.Dense(4, name='output2')(dense_output)  # Второй выходной слой с 4 нейронами

# Создание и компиляция модели
model = tf.keras.Model(inputs=inputs, outputs=[output1, output2])
model.compile(optimizer='adam', loss='mse')

# Печать сводки модели
model.summary()

print(np.max(X))  # Должно быть меньше input_dim вашего слоя встраивания
print(np.min(X))  # Должно быть больше или равно 0

# Обучение модели
model.fit(X, {'output1': y1, 'output2': y2}, epochs=10, batch_size=32, validation_split=0.2)



   0             1
0  1  1.483197e+09
1  2  1.483481e+09
2  3  1.483654e+09
3  4  1.483864e+09
4  5  1.484086e+09
(5500, 4)
(5500, 4)
